# Exploration of F1 Data
using F1 data from Ergast API (compiled by Kaggle)
[F1 World Championship (updated 2025)](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

## Introduction

This analysis aims to identify performance profiles of F1 drivers over the past two decades. 

## Methods 

SQL: `JOIN`, Common Table Expression, Temp Tables, Windows Functions, Aggregate Functions

### Step 1: Import tables from Kaggle (Python)

In [1]:
%%capture
# Use this with the F1 dataset
# Install DuckDB and kagglehub
%pip install -U jupysql duckdb-engine
%pip install kagglehub[pandas-datasets]


In [2]:
# Load packages
import kagglehub
import pandas as pd
import os
import duckdb as ddb

# Configure the SQL magic
%load_ext sql
%config SqlMagic.autopandas = True  # Return Pandas DataFrames instead of regular result sets
%config SqlMagic.feedback = False # after running query, don't produce output of "Running query in 'sqlite:///chinook.sqlite'"
%config SqlMagic.displaycon = False # Don't show connection string after executing a query

# Use %sql magic to connect to the DuckDB database
%sql duckdb:///:memory:

# Download latest data
path = kagglehub.dataset_download("rohanrao/formula-1-world-championship-1950-2020")

# Load each CSV into a DataFrame
tables = []
for f in os.listdir(path):
    if f.endswith('.csv'):
        table_name = f.replace('.csv', '')
        file_path = os.path.join(path, f)
        tables.append(table_name)
        globals()[table_name] = pd.read_csv(file_path)


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
("Names of Tables: ", list(tables))

('Names of Tables: ',
 ['circuits',
  'status',
  'lap_times',
  'sprint_results',
  'drivers',
  'races',
  'constructors',
  'constructor_standings',
  'qualifying',
  'driver_standings',
  'constructor_results',
  'pit_stops',
  'seasons',
  'results'])

In [4]:
for table_name in tables:
    df = globals()[table_name]
    print(f"\n Table: {table_name}")
    print("- Columns:", list(df.columns))


 Table: circuits
- Columns: ['circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng', 'alt', 'url']

 Table: status
- Columns: ['statusId', 'status']

 Table: lap_times
- Columns: ['raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds']

 Table: sprint_results
- Columns: ['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 'fastestLapTime', 'statusId']

 Table: drivers
- Columns: ['driverId', 'driverRef', 'number', 'code', 'forename', 'surname', 'dob', 'nationality', 'url']

 Table: races
- Columns: ['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time']

 Table: constructors
- Columns: ['constructorId', 'constructorRef', 'name', 'nationality', 'url']

 Table: constructor_standings
- Columns: ['constru

### Step 2: Explore data

#### - Look at driver-level race performance averages

In [5]:
%%sql
SELECT 
  d.forename,
  d.surname,
  AVG(q.position) AS avg_qualifying_position,
  AVG(r.positionOrder) AS avg_race_position,
  AVG(r.points) AS avg_points
FROM drivers d
JOIN results r ON d.driverId = r.driverId
LEFT JOIN qualifying q ON d.driverId = q.driverId AND r.raceId = q.raceId
LEFT JOIN races ON q.raceId = races.raceId
WHERE races.year >= 2005 
GROUP BY d.driverId, d.forename, d.surname
ORDER BY avg_points DESC
LIMIT 10

,forename,surname,avg_qualifying_position,avg_race_position,avg_points
0,Max,Verstappen,4.727273,5.645933,13.935407
1,Lewis,Hamilton,4.073034,5.019663,13.540730
2,Sebastian,Vettel,6.247492,7.090301,10.334448
3,Charles,Leclerc,6.208054,7.557047,9.147651
4,Nico,Rosberg,6.834951,8.252427,7.740291
5,Oscar,Piastri,7.543478,7.891304,7.543478
6,Lando,Norris,7.218750,7.851562,7.421875
7,Valtteri,Bottas,7.846154,8.967611,7.238866
8,Fernando,Alonso,8.173295,8.213068,6.275568
9,Mark,Webber,6.626506,8.933735,6.153614


use COALESCE() somewhere

#### - Look at qualifying order

In [6]:
%%sql
SELECT 
  d.forename,
  d.surname,
  AVG(q.position) AS avg_qualifying_position,
FROM drivers d
JOIN qualifying q ON d.driverId = q.driverId
JOIN races ON q.raceId = races.raceId
WHERE races.year >= 2005 
GROUP BY d.forename, d.surname
ORDER BY avg_qualifying_position ASC
LIMIT 10

,forename,surname,avg_qualifying_position
0,Lewis,Hamilton,4.073034
1,Max,Verstappen,4.727273
2,Charles,Leclerc,6.208054
3,Sebastian,Vettel,6.247492
4,Mark,Webber,6.626506
5,Nico,Rosberg,6.834951
6,Lando,Norris,7.218750
7,Oscar,Piastri,7.543478
8,Juan,Pablo Montoya,7.555556
9,Kimi,Räikkönen,7.665493


#### - Create TEMP table to hold race-level driver performances

In [7]:
%%sql 
DROP TABLE IF EXISTS driver_race_summary;

CREATE TEMP TABLE driver_race_summary AS
SELECT 
  r.driverId,
  r.raceId,
  r.constructorId,
  r.grid,
  r.positionOrder AS finish_position,
  r.points,
  q.position AS quali_position,
  CASE 
    WHEN r.fastestLapSpeed NOT IN ('\N', '') THEN 
      CAST(SPLIT_PART(r.fastestLapTime, ':', 1) AS INT) * 60000 +  -- minutes to ms
      CAST(SPLIT_PART(r.fastestLapTime, ':', 2) AS DOUBLE) * 1000  -- seconds to ms
    ELSE NULL
  END AS fastest_lap_time_in_ms
FROM results r
LEFT JOIN qualifying q ON r.driverId = q.driverId AND r.raceId = q.raceId
LEFT JOIN races ON q.raceId = races.raceId
WHERE races.year >= 2005 

,Success


In [8]:
%%sql 
SELECT * FROM driver_race_summary ORDER BY driverId, raceId

,driverId,raceId,constructorId,grid,finish_position,points,quali_position,fastest_lap_time_in_ms
0,1,1,1,18,20,0.0,15,89020.0
1,1,2,1,12,7,1.0,13,99141.0
2,1,3,1,9,6,3.0,9,114665.0
3,1,4,1,5,4,5.0,5,94915.0
4,1,5,1,14,9,0.0,14,83839.0
...,...,...,...,...,...,...,...,...
8261,861,1141,3,16,17,0.0,18,84296.0
8262,861,1142,3,0,14,0.0,14,96867.0
8263,861,1143,3,19,19,0.0,19,NaN
8264,861,1144,3,20,19,0.0,19,89411.0


In [9]:
%%sql 

WITH driver_averages AS (
  SELECT 
    driverId,
    COUNT(DISTINCT raceId) AS races_count,
    AVG(finish_position) AS avg_finish,
    AVG(quali_position) AS avg_quali,
    AVG(points) AS avg_points,
    AVG(
    CASE 
      WHEN fastest_lap_time_in_ms IS NOT NULL THEN fastest_lap_time_in_ms
      ELSE NULL
    END
  ) AS avg_fastest_lap_time
  FROM driver_race_summary
  GROUP BY driverId
)

SELECT 
  d.forename || ' ' || d.surname AS driver_name,
  da.*
FROM driver_averages da
JOIN drivers d ON da.driverId = d.driverId
ORDER BY avg_points DESC
LIMIT 10

,driver_name,driverId,races_count,avg_finish,avg_quali,avg_points,avg_fastest_lap_time
0,Max Verstappen,830,209,5.645933,4.727273,13.935407,88952.095477
1,Lewis Hamilton,1,356,5.019663,4.073034,13.540730,89955.959654
2,Sebastian Vettel,20,299,7.090301,6.247492,10.334448,90870.454861
3,Charles Leclerc,844,149,7.557047,6.208054,9.147651,88366.215827
4,Nico Rosberg,3,206,8.252427,6.834951,7.740291,92634.121212
5,Oscar Piastri,857,46,7.891304,7.543478,7.543478,87508.386364
6,Lando Norris,846,128,7.851562,7.218750,7.421875,87911.232000
7,Valtteri Bottas,822,247,8.967611,7.846154,7.238866,90265.719008
8,Fernando Alonso,4,352,8.213068,8.173295,6.275568,90382.772189
9,Mark Webber,17,166,8.933735,6.626506,6.153614,90939.848101


#### - Rolling average finish position for each driver 

In [10]:
%%sql
SELECT 
  d.forename || ' ' || d.surname AS driver_name,
  drs.raceId,
  drs.finish_position,
  AVG(drs.finish_position) OVER (
    PARTITION BY drs.driverId 
    ORDER BY drs.raceId 
    ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
  ) AS rolling_avg_finish
FROM driver_race_summary drs
LEFT JOIN drivers d ON drs.driverId = d.driverId
ORDER BY drs.driverId, drs.raceId;

,driver_name,raceId,finish_position,rolling_avg_finish
0,Lewis Hamilton,1,20,20.000000
1,Lewis Hamilton,2,7,13.500000
2,Lewis Hamilton,3,6,11.000000
3,Lewis Hamilton,4,4,9.250000
4,Lewis Hamilton,5,9,9.200000
...,...,...,...,...
8261,Franco Colapinto,1141,17,11.666667
8262,Franco Colapinto,1142,14,12.000000
8263,Franco Colapinto,1143,19,13.833333
8264,Franco Colapinto,1144,19,15.166667


#### - Changes in position (qualifying VS result)

In [11]:
%%sql 

WITH position_deltas AS (
  SELECT 
    driverId,
    raceId,
    quali_position,
    finish_position,
    (quali_position - finish_position) AS delta
  FROM driver_race_summary
  WHERE quali_position IS NOT NULL
)
SELECT 
  d.forename || ' ' || d.surname AS driver_name,
  COUNT(*) AS races_count,
  AVG(delta) AS avg_qualifying_delta
FROM position_deltas pd
JOIN drivers d ON pd.driverId = d.driverId
GROUP BY driver_name
ORDER BY avg_qualifying_delta DESC
LIMIT 10;

,driver_name,races_count,avg_qualifying_delta
0,Markus Winkelhock,1,5.000000
1,Karun Chandhok,11,5.000000
2,Jack Doohan,1,5.000000
3,Yuji Ide,4,4.500000
4,Luca Badoer,2,4.500000
5,Roberto Merhi,13,3.692308
6,Tiago Monteiro,37,3.621622
7,Sakon Yamamoto,21,3.476190
8,Jérôme d'Ambrosio,20,3.450000
9,Alexander Rossi,5,3.400000


## Build dataset (saved as `.sql` file)

In [16]:
%%sql 

WITH driver_race_stats AS (
    SELECT 
        r.driverId,
        r.raceId,
        r.grid,
        r.positionOrder AS finish_position,
        r.points,
        q.position AS quali_position,
        CASE 
            WHEN r.fastestLapSpeed NOT IN ('\N', '') THEN 
                CAST(SPLIT_PART(r.fastestLapTime, ':', 1) AS INT) * 60000 +  
                CAST(SPLIT_PART(r.fastestLapTime, ':', 2) AS DOUBLE) * 1000  
            ELSE NULL
        END AS fastest_lap_time_in_ms
    FROM results r
    LEFT JOIN qualifying q ON r.driverId = q.driverId AND r.raceId = q.raceId
    LEFT JOIN races ON q.raceId = races.raceId
    WHERE races.year >= 2005 
)

SELECT 
    d.driverId,
    d.forename || ' ' || d.surname AS driver_name,
    COUNT(DISTINCT drs.raceId) AS races_count,
    AVG(drs.finish_position) AS avg_finish_position,
    STDDEV(drs.finish_position) AS finish_stddev,
    AVG(drs.quali_position) AS avg_qualifying_position,
    AVG(drs.points) AS avg_points,
    AVG(drs.grid) AS avg_grid_position,
    AVG(CASE 
          WHEN drs.grid IS NOT NULL AND drs.finish_position IS NOT NULL 
          THEN CASE WHEN drs.finish_position < drs.grid THEN 1 ELSE 0 END
          ELSE NULL
        END) AS improvement_rate,
    AVG(drs.fastest_lap_time_in_ms) AS avg_fastest_lap_time_in_ms
FROM driver_race_stats drs
JOIN drivers d ON drs.driverId = d.driverId
GROUP BY d.driverId, driver_name

,driverId,driver_name,races_count,avg_finish_position,finish_stddev,avg_qualifying_position,avg_points,avg_grid_position,improvement_rate,avg_fastest_lap_time_in_ms
0,5,Heikki Kovalainen,112,13.285714,5.625599,13.883929,0.937500,13.741071,0.544643,93538.770642
1,11,Takuma Sato,55,14.890909,3.525950,16.618182,0.090909,16.400000,0.672727,88388.037736
2,7,Sébastien Bourdais,27,14.148148,4.520923,14.888889,0.222222,14.814815,0.481481,90866.000000
3,24,Vitantonio Liuzzi,81,15.913580,5.015470,17.049383,0.320988,16.555556,0.604938,92793.125000
4,34,Yuji Ide,4,17.250000,3.095696,21.750000,0.000000,20.750000,1.000000,96476.000000
...,...,...,...,...,...,...,...,...,...,...
97,839,Esteban Ocon,155,11.180645,4.715918,11.593548,2.838710,11.135484,0.522581,89888.551724
98,849,Nicholas Latifi,61,15.852459,2.803784,17.885246,0.147541,16.672131,0.573770,89439.559322
99,854,Mick Schumacher,43,15.604651,3.071680,16.279070,0.279070,15.232558,0.488372,89070.200000
100,852,Yuki Tsunoda,90,13.522222,4.133252,12.744444,0.966667,12.133333,0.366667,88706.878049
